# Title: Multi_Model_TTA_Inference_Ensemble
## Description: 학습된 여러 모델과 TTA(Test Time Augmentation)를 결합하여 추론의 안정성과 정확도를 극대화하는 앙상블 파이프라인.
## Input: 
 - models (List[nn.Module]): 추론에 사용할 사전 학습된 모델 리스트
 - test_loader (DataLoader): 추론 대상 데이터 로더
 - device (torch.device): 추론 장치 (GPU 권장)
## Output: 
 - final_probs (np.array): 모든 모델과 TTA 결과가 평균된 최종 클래스 확률 행렬
## Check Point: 
 - TTA는 추론 시간을 증가시키므로 배치 사이즈와 TTA 횟수 조절 필요.
 - `torch.cuda.amp.autocast`를 통해 추론 속도 최적화.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm
from torch.cuda.amp import autocast

# [Block 1] TTA(Test Time Augmentation) 정의
def apply_tta(imgs):
    """
    입력 이미지에 대해 여러 변환(Flip, Rotate 등)을 적용하여 반환
    (예: 원본, 좌우반전, 상하반전 등 7개 케이스)
    """
    # 0: 원본
    tta_imgs = [imgs]
    # 1-3: 회전
    tta_imgs.append(torch.rot90(imgs, 1, (2, 3)))
    tta_imgs.append(torch.rot90(imgs, 2, (2, 3)))
    tta_imgs.append(torch.rot90(imgs, 3, (2, 3)))
    # 4-5: 반전
    tta_imgs.append(torch.flip(imgs, [2])) # Vertical
    tta_imgs.append(torch.flip(imgs, [3])) # Horizontal
    # 6: 전치
    tta_imgs.append(imgs.transpose(2, 3))
    
    return torch.cat(tta_imgs, dim=0) # (Batch * 7, C, H, W)

# [Block 2] TTA 적용 추론 루프
def get_tta_probs(model, loader, device):
    """
    단일 모델에 대해 TTA를 적용하여 평균 확률값을 계산
    """
    model.to(device)
    model.eval()
    all_probs = []

    with torch.no_grad():
        for imgs in tqdm(loader, desc="Inference with TTA", leave=False):
            imgs = imgs.to(device)
            batch_size = imgs.shape[0]

            # TTA 적용 (7배 증폭)
            imgs_tta = apply_tta(imgs)

            # Mixed Precision 추론
            with autocast():
                outputs = model(imgs_tta)
                probs = F.softmax(outputs, dim=1)

            # 증폭된 결과를 다시 배치 단위로 분리하여 평균 계산
            # probs shape: (batch_size * 7, num_classes)
            # view를 통해 (7, batch_size, num_classes)로 변경 후 dim=0 평균
            avg_probs = probs.view(7, batch_size, -1).mean(dim=0)
            all_probs.append(avg_probs.cpu().numpy())

    return np.concatenate(all_probs, axis=0)

# [Block 3] Multi-Model Ensemble
def ensemble_inference(model_paths, model_arch_func, test_loader, device):
    """
    저장된 가중치들을 로드하여 다중 모델 앙상블 수행
    """
    ensemble_probs = []
    
    for path in model_paths:
        print(f"📦 Loading model from: {path}")
        # 모델 아키텍처 생성 및 가중치 로드
        model = model_arch_func()
        model.load_state_dict(torch.load(path, map_location=device))
        
        # 모델별 TTA 결과 획득
        probs = get_tta_probs(model, test_loader, device)
        ensemble_probs.append(probs)
        
        # 메모리 관리를 위해 모델 명시적 삭제
        del model
        torch.cuda.empty_cache()

    # 모든 모델의 확률값 평균 (Soft Voting)
    final_probs = np.mean(ensemble_probs, axis=0)
    return final_probs

## How to Use
1. **모델 정의 함수**: `model_arch_func`는 인자 없이 모델 객체를 반환하는 함수여야 합니다. (예: `lambda: timm.create_model('resnet50', ...)`)
2. **경로 설정**: 학습 완료된 `.pth` 파일 경로들을 리스트로 묶어 `model_paths`에 전달합니다.
3. **결과 변환**: `final_probs.argmax(axis=1)`을 통해 최종 예측 클래스를 얻을 수 있습니다.

## Troubleshooting
- **GPU Memory Error**: TTA 적용 시 배치가 7배(정의에 따라 다름) 커지므로, 추론 시에는 `batch_size`를 학습 때보다 낮게(예: 1/8 수준) 설정하세요.
- **TTA 조합 변경**: `apply_tta` 함수 내의 리스트를 수정하여 `Flip`만 사용하거나 `Rotate`만 사용하는 등 자유롭게 커스텀 가능합니다. (단, `view(N, ...)`의 N 값과 일치시켜야 함)